## Com base nos problemas estudados no Trabalho1, escolher um problema da categoria NP da sua preferência e realize as seguintes tarefas:
Eu esolhi solucionar o clássico problema do caixeiro viajante. Especificamente inpirado no dataset [ca4636](https://www.math.uwaterloo.ca/tsp/world/calog.html) que trás 4.636 cidades canadenses com o objetivo de encontrar o caminho mais curto por todas elas. Aqui consideramos todos os nós como interligados com a distância eucliadiana como o custo de cada aresta entre duas cidades. 



#### i) Propor e implementar uma heurística construtiva para achar uma solução inicial para o problema escolhido. Descrever a representação da sua solução e seu método construtivo e implemente.
Considerando que todas as cidades são virtualmente acessíveis entre si, como método construtivo utilizaei apenas uma permutação aleatória das cidades para construir um caminho inicial. A representação das cidades se dá por um dicionário com índices como chave e o seu par de coordenadas como valor. ex: {1 : [41800.0000, 82650.0000], 2 : [41966.6667, 82533.3333], 3 : [41983.3333, 82933.3333]}.
Já o caminho é representado pela lista chamada "path". 



### ii) Propor e implementar uma heurística de busca local para achar uma solução de melhor qualidade que a encontrada no item i). Descrever o mecanismo de busca local adotado e implemente.
Como algoritmo de busca local eu utilizei o Simulated Annealing usando a técnica 2-opt para variar a solução, e resfriamento linear. Ou seja, a partir de uma solução corrente o algoritmo sorteia dois índices para inverter as cidades entre eles. Mas antes de inverter de fato, a função "swap_cost" estima a diferença de custo dessa mudança, caso ela reduza o custo do caminho a mudança é efetuada, caso contrário sorteamos a probabilidade de aceitar essa solução pior por meio da temperatura e amplitude da piora. Esse processo continua até que uma das condições de parada seja atingida: temperatura atinge o zero, número máximo de iterações, ou número máximo de pioras consecutivas.

## Implementação

In [3]:
import tsplib95 
import networkx as nx
import numpy as np
np.random.seed(42)


class TSPSolver:
    def solve(self,nodes):
        self.nodes = nodes
        initial_solution = self.initial_solution()
        best_solution = self.local_search(initial_solution)

    def initial_solution(self):
        n = len(self.nodes)
        path = np.random.permutation(np.arange(1,n+1,1)) 

        cost = self.cost(path)
        solution = {"path":path} | cost
        return solution

    def local_search(self, current_solution):
        self.simulated_annealing(current_solution)

    def simulated_annealing(self, current_solution):
        current_path = current_solution["path"]
        current_eval = current_solution["total_cost"]
        initial_eval = current_eval
        cooling_rate = 1
        max_non_improov = 1_000
        max_iterations = 50_000
        temperature = 100_000

        n = len(current_path)
        iteration = 0
        non_improov = 0

        while non_improov < max_non_improov and temperature > 1 and iteration < max_iterations:
            iteration += 1
            i,j = np.random.choice(range(1,n-1),size=2,replace=False)
            bounds = (i,j) if i<j else (j,i)
            swap_cost = self.swap_cost(current_path,bounds)

            # print(self.check_eval(current_eval,current_path, swap_cost))
            if swap_cost < 0:
                non_improov = 0
                current_path = self.swap_n_reverse(current_path, bounds)
                current_eval += swap_cost
            else:
                non_improov += 1
                accepting_prob = np.exp(-swap_cost/temperature)
                if np.random.uniform() < accepting_prob:
                    current_path = self.swap_n_reverse(current_path, bounds)
                    current_eval += swap_cost
                
                temperature -= cooling_rate

    
        print(f"iteration: {iteration}, Non-improvments: {non_improov}, temperature: {temperature}")
        print(f"initial cost:{initial_eval}; current cost: {current_eval}; {current_eval/initial_eval}") #% \n ({i,j})")
        return {"path": current_path, "cost": current_eval}

    def node_distance(self,a,b):
        distance = self.distance(self.nodes[a], self.nodes[b])
        return distance

    def distance(self, a, b):
        #todo: check performance
        return np.sqrt(
        (a[0] - b[0])**2 + (a[1] - b[1])**2
        )

        
    def swap_n_reverse(self, path, bounds):
        new_path = np.copy(path)
        i,j = bounds
        new_path[i:j] = new_path[i:j][::-1]

        return new_path


    def check_eval(self, current_eval,current_path, swap_cost, bounds):
        calculated_cost = self.cost(current_path)["total_cost"]
        diff = calculated_cost - current_eval + 0.0000001
        print(f"i,j: {bounds}")
        print(f"\n swap/diff: {swap_cost/diff} \n eval: {current_eval}; calculated: {calculated_cost}; diff: {diff} diff% {current_eval/calculated_cost}")

        return ((calculated_cost - current_eval)**2)**0.5 < 100


    def swap_cost(self, path, bounds):
        i,j = bounds
        n1,n2 = path[i-1], path[i]
        n3,n4 = path[j-1], path[j]

        d = self.node_distance
        cost = - ((d(n1,n2) - d(n1,n3)) + (d(n3,n4) - d(n2,n4)))

        return cost
            
  
    def cost(self, path):
        n = len(path)
        cost = np.array([self.distance(self.nodes[path[i]],self.nodes[path[i+1]]) for i in range(n-1)])
        total_cost = sum(cost)

        return {"cost":cost, "total_cost":total_cost}

solver = TSPSolver()

# Teste Pequeno

# Testes em Cases

## [ca4636](https://www.math.uwaterloo.ca/tsp/world/calog.html) - 4.636 Cidades Canadenses 
![image](./capoints.gif)

In [5]:

canada = tsplib95.load("ca4663.tsp")
ca_nodes = canada.node_coords

ca_path = solver.solve(ca_nodes)

i: 343, Non-improvments: 1, temperature: 1
initial cost:129178026.90957189; current cost: 49091107.324098006; 0.3800267622794944


| Solução inicial | Melhor solução encontrada  | Solução ótima |
|-----------------|----------------------------|---------------|
| 129.178.027     | 19.265.493                 | 1.290.319     |

## [FI10639](https://www.math.uwaterloo.ca/tsp/world/calog.html) - 10.639 Cidades Finlandesas  
<!-- ![image](./fipoints.gif) -->
<img src="fipoints.gif" alt="drawing" width="400"/>

In [17]:

finland = tsplib95.load("./fi10639.tsp")
fi_nodes = finland.node_coords

fi_path = solver.solve(fi_nodes)

i: 7884, Non-improvments: 8, temperature: 0.999999333559433
initial cost:42685168.77336324; current cost: 21737301.835322652; 0.5092471802263869


| Solução inicial | Melhor solução encontrada  | Solução ótima |
|-----------------|----------------------------|---------------|
| 42.643.565      | 21.737.301                 |  557.315      |

## [IT16862](https://www.math.uwaterloo.ca/tsp/world/calog.html) - 16.862 Cidades Italianas
![image](./itpoints.gif)

In [18]:

italy = tsplib95.load("./it16862.tsp")
it_nodes = italy.node_coords

it_path = solver.solve(it_nodes)

i: 14637, Non-improvments: 3, temperature: 0.999999333559433
initial cost:70298389.01041465; current cost: 38436309.03375622; 0.5467594574331129


| Solução inicial | Melhor solução encontrada  | Solução ótima |
|-----------------|----------------------------|---------------|
| 70.527.061      | 38.436.309                 | 557.315       |